In [1]:
# Import dependencies
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import json
import pandas as pd
from pandas.io import sql
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from consts import *
%matplotlib inline

In [2]:
# Connecting to Postgres instance
engine = create_engine(CREATE_ENGINE_STR)

In [3]:
# Printing info for table names 
print (engine.table_names())

['education', 'committee_summary_2020', 'agg_county_votes', 'fec_donor_az', 'health_metrics', 'birth_death_rate', 'postal_codes', 'fec_donor_mi', 'fec_donor_wi', 'fec_committee', 'fec_donor_pa', 'donations', 'pres_votes_6t', 'unemployment', 'fec_donor_nc', 'fec_donor_fl']


In [4]:
def calculate_election_yr(i):
    return (2000 + i*4)

In [5]:
# Definition to take in the votes data frame. Function takes in the votes dataframe with 20 years of data. 
# This will loop thru every 4 years, runs thru all of the county votes then return it in a list. 
# This will aggregate everything and return a list in a df
def get_votes_intervals(votes_df, state_po):
    votes_states_df = votes_df[votes_df['state_po']==state_po]
    i = ELECTION_STARTING_YR
    
    four_yr_dfs = []
    while (i <= ELECTION_ENDING_YR):
        votes_states_interval_df = votes_states_df[votes_states_df['year']==i]    
        four_yr_dfs.append(votes_states_interval_df)
        i += ELECTION_INTERVAL
    
    return four_yr_dfs

In [6]:
# Goes thru each county (string), to pull the election date and calculate votes in the county that are democrat (blue), republic (red) and other. 
def vote_distribution(county, election_df, state, i):        
    county_df = election_df[election_df['county']==county]
    county_blue_df = county_df[county_df['party']==MAJOR_PARTIES[0]]
    county_red_df = county_df[county_df['party']==MAJOR_PARTIES[1]]  
    
    #Other = not democratic AND not republican  
    other_votes = 0
    blue_votes = pd.to_numeric(county_blue_df["candidatevotes"].sum(), errors='coerce')
    red_votes = pd.to_numeric(county_red_df["candidatevotes"].sum(), errors='coerce')
    
    unique_parties = county_df['party'].unique()
    for party in unique_parties:
        #Get a sum of all non major parties for other category
        if party not in MAJOR_PARTIES:
            party_df = county_df[county_df['party']==party]
            other_votes += pd.to_numeric(party_df["candidatevotes"].sum(), errors='coerce')
    
    #Total votes it the sum of blue + red + other
    total_votes = blue_votes + red_votes + other_votes
    
    #Get the respective percentages
    percent_blue = (blue_votes / total_votes)
    percent_red = (red_votes / total_votes)
    percent_other = (other_votes / total_votes)
    
    election_year = calculate_election_yr(i)
    
    #Set the unemployment data points from the county for that election year.
    unemployment_sql = f'SELECT * FROM unemployment WHERE "County" = \'{county}\' AND "Stabr" = \'{state}\''
    unemployment_df = pd.read_sql_query(unemployment_sql,con=engine)
    
    #Get POPPCT_URBAN -> urban_pct
    unemployment_col = "Unemployment_rate_" + str(election_year)  
    urban_pct = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPPCT_URBAN'].values[0]      
    unemployment_rate = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), unemployment_col].values[0]
    urban_den = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPDEN_URBAN'].values[0]
    rural_pct = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPPCT_RURAL'].values[0]
    rural_den = unemployment_df.loc[(unemployment_df['County'] == county) & (unemployment_df["Stabr"] == state), 'POPDEN_RURAL'].values[0]
    
    county_tuple = (
        blue_votes,
        red_votes,
        other_votes,
        total_votes,
        percent_blue,
        percent_red,
        percent_other,
        county,
        state,
        election_year,
        urban_pct,
        unemployment_rate,
        urban_den,
        rural_pct,
        rural_den
    )
    return county_tuple

In [7]:
def donor_distribution(election_df, county, state, i):    
    county = county.strip()
        
    county_df = election_df[election_df['county']==county]
    county_blue_df = county_df[county_df['party']==MAJOR_PARTIES[0]]
    county_red_df = county_df[county_df['party']==MAJOR_PARTIES[1]]  
    
    #Other = not democratic AND not republican  
    other_amt = 0
    blue_amt = pd.to_numeric(county_blue_df["transaction_amt"].sum(), errors='coerce')
    red_amt = pd.to_numeric(county_red_df["transaction_amt"].sum(), errors='coerce')
    
    unique_parties = county_df['party'].unique()
    for party in unique_parties:
        #Get a sum of all non major parties for other category
        if party not in MAJOR_PARTIES:
            party_df = county_df[county_df['party']==party]
            other_amt += pd.to_numeric(party_df["transaction_amt"].sum(), errors='coerce')
    
    #Total transaction amount it the sum of blue + red + other
    total_amt = blue_amt + red_amt + other_amt
    
    #Get the respective percentages
    percent_blue = (blue_amt / total_amt)
    percent_red = (red_amt / total_amt)
    percent_other = (other_amt / total_amt)
    
    election_year = calculate_election_yr(i)
    donor_tuple = (
        blue_amt,
        red_amt,
        other_amt,
        total_amt,
        percent_blue,
        percent_red,
        percent_other,
        county,
        state,
        election_year
    )
    return donor_tuple

In [8]:
# Run every election year in the loop, get election df, look for the vote distribution
# Will be run on a single state and will return a dictionary tha tcountains every county in the state as a key. 
# The value (number of votes) is a dictionary of values.
def county_vote_distribution(four_yr_dfs, state):
    #Loop through each election DF
    for i in range(len(four_yr_dfs)):
        election_df = four_yr_dfs[i]
        unique_counties = election_df["county"].unique()
        print(f"Running county election year: {calculate_election_yr(i)} num countines: {len(unique_counties)}")
        
        #Loop through each unique county
        for county in unique_counties:
            print(f"Running county: {county}")
            #Get the percent of the vote distribution for that county
            county_tuple = vote_distribution(county, election_df, state, i)            
            county_votes_df = pd.DataFrame([county_tuple], columns=VOTES_COLS)
            #print("county_votes_df")
            #print(county_votes_df.size)
            #print(county_votes_df.head())
            #Write the vote tallies per county to DB
            county_votes_df.to_sql(TABLE_AGG_VOTES, con=engine, if_exists="append")

In [9]:
"""
ACE	Ace Party	
AKI	Alaskan Independence Party	
AIC	American Independent Conservative	
AIP	American Independent Party	
AMP	American Party	
APF	American People's Freedom Party	
AE	Americans Elect	
CIT	Citizens' Party	
CMD	Commandments Party	
CMP	Commonwealth Party of the U.S.	
COM	Communist Party	
CNC	Concerned Citizens Party Of Connecticut	
CRV	Conservative Party	
CON	Constitution Party	
CST	Constitutional	
COU	Country	
DCG	D.C. Statehood Green Party	
DNL	Democratic -Nonpartisan League	
DEM	Democratic Party	
D/C	Democratic/Conservative	
DFL	Democratic-Farmer-Labor	
DGR	Desert Green Party	
FED	Federalist	
FLP	Freedom Labor Party	
FRE	Freedom Party	
GWP	George Wallace Party	
GRT	Grassroots	
GRE	Green Party	
GR	Green-Rainbow	
HRP	Human Rights Party	
IDP	Independence Party	
IND	Independent	
IAP	Independent American Party	
ICD	Independent Conservative Democratic	
IGR	Independent Green	
IP	Independent Party	
IDE	Independent Party of Delaware	
IGD	Industrial Government Party	
JCN	Jewish/Christian National	
JUS	Justice Party	
LRU	La Raza Unida	Also see RUP
LBR	Labor Party	Also see LAB
LFT	Less Federal Taxes	
LBL	Liberal Party	
LIB	Libertarian Party	
LBU	Liberty Union Party	
MTP	Mountain Party	
NDP	National Democratic Party	
NLP	Natural Law Party	
NA	New Alliance	
NJC	New Jersey Conservative Party	
NPP	New Progressive Party	
NPA	No Party Affiliation	
NOP	No Party Preference	Commonly used in CA & WA
NNE	None	
N	Nonpartisan	
NON	Non-Party	
OE	One Earth Party	
OTH	Other	
PG	Pacific Green	
PSL	Party for Socialism and Liberation	
PAF	Peace And Freedom	Also see PFP
PFP	Peace And Freedom Party	Also see PAF
PFD	Peace Freedom Party	
POP	People Over Politics	
PPY	People's Party	
PCH	Personal Choice Party	
PPD	Popular Democratic Party	
PRO	Progressive Party	
NAP	Prohibition Party	
PRI	Puerto Rican Independence Party	
RUP	Raza Unida Party	Also see LRU
REF	Reform Party	
REP	Republican Party	
RES	Resource Party	
RTL	Right To Life	
SEP	Socialist Equality Party	
SLP	Socialist Labor Party	
SUS	Socialist Party	
SOC	Socialist Party U.S.A.	
SWP	Socialist Workers Party	
TX	Taxpayers	
TWR	Taxpayers Without Representation	
TEA	Tea Party	
THD	Theo-Democratic	
LAB	U.S. Labor Party	Also see LBR
USP	U.S. People's Party	
UST	U.S. Taxpayers Party	
UN	Unaffiliated	
UC	United Citizen	
UNI	United Party	
UNK	Unknown	
VET	Veterans Party	
WTP	We the People	
W	Write-In
"""

"\nACE\tAce Party\t\nAKI\tAlaskan Independence Party\t\nAIC\tAmerican Independent Conservative\t\nAIP\tAmerican Independent Party\t\nAMP\tAmerican Party\t\nAPF\tAmerican People's Freedom Party\t\nAE\tAmericans Elect\t\nCIT\tCitizens' Party\t\nCMD\tCommandments Party\t\nCMP\tCommonwealth Party of the U.S.\t\nCOM\tCommunist Party\t\nCNC\tConcerned Citizens Party Of Connecticut\t\nCRV\tConservative Party\t\nCON\tConstitution Party\t\nCST\tConstitutional\t\nCOU\tCountry\t\nDCG\tD.C. Statehood Green Party\t\nDNL\tDemocratic -Nonpartisan League\t\nDEM\tDemocratic Party\t\nD/C\tDemocratic/Conservative\t\nDFL\tDemocratic-Farmer-Labor\t\nDGR\tDesert Green Party\t\nFED\tFederalist\t\nFLP\tFreedom Labor Party\t\nFRE\tFreedom Party\t\nGWP\tGeorge Wallace Party\t\nGRT\tGrassroots\t\nGRE\tGreen Party\t\nGR\tGreen-Rainbow\t\nHRP\tHuman Rights Party\t\nIDP\tIndependence Party\t\nIND\tIndependent\t\nIAP\tIndependent American Party\t\nICD\tIndependent Conservative Democratic\t\nIGR\tIndependent Green\t\

In [10]:
def map_zip_county(unique_zips, state_zips):
    county_dict = {}
    unique_counties = {}
    for zipcode in unique_zips:
        #Filter out on the zip code from the state_zips DF
        county_zip = state_zips[state_zips["zip"] == zipcode]
        #Get the county name from the DF and convert it to lower
        county_name = county_zip["county"].to_string(index=False).strip().lower()
        #Filter out the county string within
        county_name = county_name.replace(" county", "").capitalize()
        
        county_dict[zipcode] = county_name
        if county_name not in unique_counties:
            unique_counties[county_name] = True
            
    return (county_dict, unique_counties.keys())

In [11]:
def select_columns(df, column_names):
    new_frame = df.loc[:, column_names]
    return new_frame

In [12]:
def one_hot_encode(df):
    # Generate our categorical variable list
    cat_vars = df.dtypes[df.dtypes == "object"].index.tolist()

    # Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(df[cat_vars]))

    # Add the encoded variable names to the DataFrame
    encode_df.columns = enc.get_feature_names(cat_vars)
    
    return encode_df

In [13]:
def label_enc(df):
    # Create encoder
    le = LabelEncoder()
    # Encode first DataFrame 1 (where all values are floats)
    df = df.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
    return df

In [14]:
#Add a new column party to the DF that maps the committee party abbreviation to a major party
def merge_cmtid_party(donor_df):        
    #Get the major party strings to map to 
    party_repub = MAJOR_PARTIES[1]
    party_democrat = MAJOR_PARTIES[0]
    party_other = "other"
    
    #Map the affiliation code to the party affiliation
    cmte_party_map = {
        "REP": party_repub,
        "TEA": party_repub,
        "DNL": party_democrat,
        "DNL": party_democrat,
        "DEM": party_democrat,
        "D/C": party_democrat,
        "DFL": party_democrat,
        "THD": party_democrat,
        "PPD": party_democrat,
        "UNK": party_other
    }
    
    donor_df["party"] = donor_df["cmte_pty_affiliation"].map(cmte_party_map)
    
    return donor_df

In [15]:
#Loop through each of the election year DFs and 
def donation_county_cycle_distribution(four_yr_dfs, state_zips, committee_df, state):
    #Loop through each election year DF
    for i in range(len(four_yr_dfs)):
        election_df = four_yr_dfs[i]
        #Without zipcode can't do a county lookup, so drop all null values
        election_df.dropna(subset=["zip"], inplace=True)
        #Get the unique values of zip code in the election DF
        unique_zips = election_df["zip"].unique()
        #Createa a map of zip to county, and a list of all unique counties in that state
        (zip_county_map, unique_counties) = map_zip_county(unique_zips, state_zips)
        #Map the zipcode to the county name per the map function
        election_df["county"] = election_df["zip"].map(zip_county_map)
        
        print(f"Running donor distribution election yr: {calculate_election_yr(i)} num countines: {len(unique_counties)}")

        #Loop through each unique county
        for county in unique_counties:
            print(f"Running county: {county}")
            #Get the donor distribution for that county, state, election year as a tuple
            donor_tuple = donor_distribution(election_df, county, state, i)
            #Create a DF to store the county donor info
            donor_df = pd.DataFrame([donor_tuple], columns = DONOR_COLS)  
            #Write the donation amounts to the DB
            donor_df.to_sql(TABLE_AGG_DONORS, con=engine, if_exists="append")

In [16]:
def str_dt(donor_date_str):
    #01/01/1996 - 12/31/1999
    donor_date = datetime.strptime(donor_date_str, '%Y-%m-%d')
    return donor_date

In [17]:
def get_year_from_date_str(donor_date_str):
    donor_date = str_dt(donor_date_str)
    donor_year = donor_date.year
    return donor_year

In [18]:
def get_donors_intervals(donor_df, state):
    donors_states_df = donor_df[donor_df['state']==state.lower()]
    
    i = ELECTION_STARTING_YR
    prev_year = ELECTION_STARTING_YR - ELECTION_INTERVAL
    ending_yr = ELECTION_ENDING_YR
    
    four_yr_dfs = []
        
    while (i <= ELECTION_ENDING_YR):
        votes_states_interval_df = donors_states_df[(donors_states_df['transaction_dt']>datetime.date(prev_year,1,1)) & (donors_states_df['transaction_dt']<datetime.date(i,3,1))]          
        four_yr_dfs.append(votes_states_interval_df)
        i += ELECTION_INTERVAL
        prev_year += ELECTION_INTERVAL
        
    return four_yr_dfs

In [19]:
#Get all donation records for a single state and return it in a dataframe
def donor_state_query(state, engine):
    #Run queries to get all donation records from the states into dfs
    donor_table_name = '"fec_donor_{}"'.format(state.lower())    
    donor_select_sql = 'select * from {}'.format(donor_table_name)
    donor_df = pd.read_sql_query(donor_select_sql,con=engine)
    return donor_df

In [20]:
#Aggregate tables are the output of this script, drop them to start fresh
def drop_agg_tables():
    sql.execute('DROP TABLE IF EXISTS %s'%TABLE_AGG_DONORS, engine)
    sql.execute('DROP TABLE IF EXISTS %s'%TABLE_AGG_VOTES, engine)

In [21]:
#Main Loop of the program
def main():
    #Read the various tables into DFs
    health_df = pd.read_sql_query('select * from "health_metrics"',con=engine)
    committee_df = pd.read_sql_query('select * from "fec_committee"',con=engine)
    votes_df = pd.read_sql_query('select * from "pres_votes_6t"',con=engine)
    zips_df = pd.read_sql_query('select * from "postal_codes"',con=engine)
    
    #Lowercase the column
    committee_df['cmte_id'] = committee_df['cmte_id'].str.lower()
    
    #Drop the aggregate tables to do fresh data analysis
    drop_agg_tables()

    #Loop through each state
    for state in SWING_STATES:
        print("State:" + state)
        print("Aggregating Vote and Donation records...")
        #Get the votes related to that state
        votes_intervals_df = get_votes_intervals(votes_df, state)

        #Get the distribution of Red, Blue, and Other votes in a list of dict per election yr e.g. 2000 + 4n
        county_vote_distribution(votes_intervals_df, state)
        
        #DF that has all donation records for a state
        donor_df_orig = donor_state_query(state, engine)
        
        #Add party column to donor data frame
        donor_df = committee_df.merge(donor_df_orig, left_on='cmte_id', right_on='cmt_id')
        donor_df = merge_cmtid_party(donor_df)
        
        #Get a list of DFs per election year per state
        donors_intervals_df = get_donors_intervals(donor_df, state)
        #Filter out the zips DF by the state
        state_zips = zips_df[zips_df["state"] == state]
                
        #Get list of tuples 
        donation_county_cycle_distribution(donors_intervals_df, state_zips, committee_df, state)
        
        print("\n")
    
    print("Swing States Aggregation Done!")

In [ ]:
#Run the main loop
main()

State:AZ
Aggregating Vote and Donation records...
Running county election year: 2000 num countines: 15
Running county: Apache
SELECT * FROM unemployment WHERE "County" = 'Apache' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr  County          area_name  Civilian_labor_force_2000   \
0     4001    AZ  Apache  Apache County, AZ                      19,028   

   Employed_2000   Unemployed_2000   Unemployment_rate_2000  \
0          17,318             1,710                     9.0   

   Civilian_labor_force_2001   Employed_2001   ... POP_UC  POPPCT_UC  \
0                      19,191          17,362  ...  18551      25.94   

    AREA_UC AREAPCT_UC POPDEN_UC  POP_RURAL POPPCT_RURAL   AREA_RURAL  \
0  44457803       0.15    1080.7      52967        74.06  28956986022   

  AREAPCT_RURAL  POPDEN_RURAL  
0         99.85           4.7  

[1 rows x 115 columns]
urban_pct 25.94
Running county: Cochise
SELECT * FROM unemployment WHERE "County" = 'Cochise' AND "Stabr" = 'AZ'
unemployment_df

Running county: Pinal
SELECT * FROM unemployment WHERE "County" = 'Pinal' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr County         area_name  Civilian_labor_force_2000   \
0     4021    AZ  Pinal  Pinal County, AZ                      69,618   

   Employed_2000   Unemployed_2000   Unemployment_rate_2000  \
0          66,450             3,168                     4.6   

   Civilian_labor_force_2001   Employed_2001   ... POP_UC  POPPCT_UC  \
0                      71,918          68,013  ...  93365      24.85   

    AREA_UC AREAPCT_UC POPDEN_UC  POP_RURAL POPPCT_RURAL   AREA_RURAL  \
0  79677947       0.57    3034.9      82311         21.9  13586206050   

  AREAPCT_RURAL  POPDEN_RURAL  
0         97.76          15.7  

[1 rows x 115 columns]
urban_pct 78.1
Running county: Santa Cruz
SELECT * FROM unemployment WHERE "County" = 'Santa Cruz' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr      County              area_name  \
0     4023    AZ  Santa Cruz  Santa Cruz County,

Running county: Maricopa
SELECT * FROM unemployment WHERE "County" = 'Maricopa' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr    County            area_name  Civilian_labor_force_2000   \
0     4013    AZ  Maricopa  Maricopa County, AZ                   1,598,108   

   Employed_2000   Unemployed_2000   Unemployment_rate_2000  \
0       1,546,411            51,697                     3.2   

   Civilian_labor_force_2001   Employed_2001   ... POP_UC  POPPCT_UC  \
0                   1,655,742       1,586,141  ...  40771       1.07   

    AREA_UC AREAPCT_UC POPDEN_UC  POP_RURAL POPPCT_RURAL   AREA_RURAL  \
0  58843870       0.25    1794.5      90229         2.36  20731564247   

  AREAPCT_RURAL  POPDEN_RURAL  
0          87.0          11.3  

[1 rows x 115 columns]
urban_pct 97.64
Running county: Mohave
SELECT * FROM unemployment WHERE "County" = 'Mohave' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr  County          area_name  Civilian_labor_force_2000   \
0     4015    AZ 

Running county: Gila
SELECT * FROM unemployment WHERE "County" = 'Gila' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr County        area_name  Civilian_labor_force_2000   \
0     4007    AZ   Gila  Gila County, AZ                      20,334   

   Employed_2000   Unemployed_2000   Unemployment_rate_2000  \
0          19,267             1,067                     5.2   

   Civilian_labor_force_2001   Employed_2001   ... POP_UC  POPPCT_UC  \
0                      20,408          19,215  ...  31589      58.94   

    AREA_UC AREAPCT_UC POPDEN_UC  POP_RURAL POPPCT_RURAL   AREA_RURAL  \
0  50348625       0.41    1625.0      22008        41.06  12272640093   

  AREAPCT_RURAL  POPDEN_RURAL  
0         99.59           4.6  

[1 rows x 115 columns]
urban_pct 58.94
Running county: Graham
SELECT * FROM unemployment WHERE "County" = 'Graham' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr  County          area_name  Civilian_labor_force_2000   \
0     4009    AZ  Graham  Graham County

Running county: Yuma
SELECT * FROM unemployment WHERE "County" = 'Yuma' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr County        area_name  Civilian_labor_force_2000   \
0     4027    AZ   Yuma  Yuma County, AZ                      64,337   

   Employed_2000   Unemployed_2000   Unemployment_rate_2000  \
0          53,817            10,520                    16.4   

   Civilian_labor_force_2001   Employed_2001   ... POP_UC  POPPCT_UC  \
0                      65,360          54,338  ...  41079      20.99   

    AREA_UC AREAPCT_UC POPDEN_UC  POP_RURAL POPPCT_RURAL   AREA_RURAL  \
0  28992500        0.2    3669.7      20416        10.43  14103190623   

  AREAPCT_RURAL  POPDEN_RURAL  
0         98.75           3.7  

[1 rows x 115 columns]
urban_pct 89.57
Running county election year: 2012 num countines: 15
Running county: Apache
SELECT * FROM unemployment WHERE "County" = 'Apache' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr  County          area_name  Civilian_labor_f

Running county: Pima
SELECT * FROM unemployment WHERE "County" = 'Pima' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr County        area_name  Civilian_labor_force_2000   \
0     4019    AZ   Pima  Pima County, AZ                     409,776   

   Employed_2000   Unemployed_2000   Unemployment_rate_2000  \
0         394,635            15,141                     3.7   

   Civilian_labor_force_2001   Employed_2001   ... POP_UC  POPPCT_UC  \
0                     414,956         396,750  ...  72070       7.35   

     AREA_UC AREAPCT_UC POPDEN_UC  POP_RURAL POPPCT_RURAL   AREA_RURAL  \
0  124127808       0.52    1503.8      73750         7.52  22768580644   

  AREAPCT_RURAL  POPDEN_RURAL  
0         95.69           8.4  

[1 rows x 115 columns]
urban_pct 92.48
Running county: Pinal
SELECT * FROM unemployment WHERE "County" = 'Pinal' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr County         area_name  Civilian_labor_force_2000   \
0     4021    AZ  Pinal  Pinal County, AZ

Running county: La Paz
SELECT * FROM unemployment WHERE "County" = 'La Paz' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr  County          area_name  Civilian_labor_force_2000   \
0     4012    AZ  La Paz  La Paz County, AZ                       7,623   

   Employed_2000   Unemployed_2000   Unemployment_rate_2000  \
0           7,140               483                     6.3   

   Civilian_labor_force_2001   Employed_2001   ... POP_UC  POPPCT_UC  \
0                       7,026           6,554  ...   8948      43.67   

    AREA_UC AREAPCT_UC POPDEN_UC  POP_RURAL POPPCT_RURAL   AREA_RURAL  \
0  15576728       0.13    1487.8      11541        56.33  11638407613   

  AREAPCT_RURAL  POPDEN_RURAL  
0         99.87           2.6  

[1 rows x 115 columns]
urban_pct 43.67
Running county: Maricopa
SELECT * FROM unemployment WHERE "County" = 'Maricopa' AND "Stabr" = 'AZ'
unemployment_df
   FIPStxt Stabr    County            area_name  Civilian_labor_force_2000   \
0     4013    AZ  Mar

Running county: Alger
SELECT * FROM unemployment WHERE "County" = 'Alger' AND "Stabr" = 'MI'
unemployment_df
   FIPStxt Stabr County         area_name  Civilian_labor_force_2000   \
0    26003    MI  Alger  Alger County, MI                       4,391   

   Employed_2000   Unemployed_2000   Unemployment_rate_2000  \
0           4,150               241                     5.5   

   Civilian_labor_force_2001   Employed_2001   ... POP_UC  POPPCT_UC  AREA_UC  \
0                       4,422           4,139  ...   2972      30.96  8501454   

  AREAPCT_UC POPDEN_UC  POP_RURAL POPPCT_RURAL  AREA_RURAL AREAPCT_RURAL  \
0       0.36     905.4       6629        69.04  2361517588         99.64   

   POPDEN_RURAL  
0           7.3  

[1 rows x 115 columns]
urban_pct 30.96
Running county: Allegan
SELECT * FROM unemployment WHERE "County" = 'Allegan' AND "Stabr" = 'MI'
unemployment_df
   FIPStxt Stabr   County           area_name  Civilian_labor_force_2000   \
0    26005    MI  Allegan  Allegan 